In [1]:
from os import listdir


In [2]:
base_dir_loc = 'fgo_image_datasets'
task_names = ['turncounter','cards','attack']

task_node_dict = {'attack':['attack','not_attack'],
                  'cards':['quick','arts','buster'],
                  'turncounter':['one','two','three']}

task_list = listdir(base_dir_loc)

In [3]:
def remove_ds_store(base_list):
    if '.DS_Store' in base_list:
        base_list.remove('.DS_Store')
    return base_list

remove_ds_store(task_list)

['attack', 'cards', 'turncounter']

In [5]:
import pandas as pd


for task in task_list:
    task_file_list = []
    task_label_list = []
    
    individual_task_labels = task_node_dict[task]
    for individual_task in individual_task_labels:
        task_path = 'fgo_image_datasets'+'/'+task+'/'+individual_task
        individual_task_list = listdir(task_path)
        task_label_name = [individual_task]*len(individual_task_list)
        
        individual_task_full_path = []
        for individual_file in individual_task_list:
            individual_loc = task_path+'/'+individual_file
            individual_task_full_path.append(individual_loc)

        task_file_list+= individual_task_full_path
        task_label_list += task_label_name
        print(len(individual_task_list),len(task_label_name))
    dat = pd.DataFrame({'file_path':task_file_list,'label':task_label_list})
    dat.to_csv(task+'.csv',index=False)
    
        


71 71
159 159
52 52
52 52
49 49
74 74
72 72
60 60


In [6]:
attack_dat = pd.read_csv('attack.csv')
cards_dat = pd.read_csv('cards.csv')
turncounter_dat = pd.read_csv('turncounter.csv')

In [7]:
attack_dat["label"] = attack_dat["label"].astype('category')
attack_dat["label_cat"] = attack_dat["label"].cat.codes

attack_dat.head()

,file_path,label,label_cat
0,fgo_image_datasets/attack/attack/a1.JPG,attack,0
1,fgo_image_datasets/attack/attack/a10.JPG,attack,0
2,fgo_image_datasets/attack/attack/a11.JPG,attack,0
3,fgo_image_datasets/attack/attack/a12.JPG,attack,0
4,fgo_image_datasets/attack/attack/a13.JPG,attack,0


In [8]:
cards_dat["label"] = cards_dat["label"].astype('category')
cards_dat["label_cat"] = cards_dat["label"].cat.codes

cards_dat.head()

,file_path,label,label_cat
0,fgo_image_datasets/cards/quick/Capture.JPG,quick,2
1,fgo_image_datasets/cards/quick/q10.JPG,quick,2
2,fgo_image_datasets/cards/quick/q11.JPG,quick,2
3,fgo_image_datasets/cards/quick/q12.JPG,quick,2
4,fgo_image_datasets/cards/quick/q13.JPG,quick,2


In [9]:
turncounter_dat["label"] = turncounter_dat["label"].astype('category')
turncounter_dat["label_cat"] = turncounter_dat["label"].cat.codes

turncounter_dat.head()

,file_path,label,label_cat
0,fgo_image_datasets/turncounter/one/1.JPG,one,0
1,fgo_image_datasets/turncounter/one/10.JPG,one,0
2,fgo_image_datasets/turncounter/one/11.JPG,one,0
3,fgo_image_datasets/turncounter/one/12.JPG,one,0
4,fgo_image_datasets/turncounter/one/13.JPG,one,0


In [10]:
turncounter_dat["label_cat"].value_counts()

0    74
2    72
1    60
Name: label_cat, dtype: int64

In [11]:
import numpy as np

def train_test_split(df,train_percent):
    msk = np.random.rand(len(df)) < train_percent
    train = df[msk]
    test = df[~msk]
    
    return train,test

train_attack,test_attack = train_test_split(attack_dat,.9)
print(train_attack.shape,test_attack.shape)

train_cards,test_cards = train_test_split(cards_dat,.9)
print(train_cards.shape,test_cards.shape)

train_turncounter,test_turncounter = train_test_split(turncounter_dat,.9)
print(train_turncounter.shape,test_turncounter.shape)

train_attack = train_attack.reset_index()
test_attack = test_attack.reset_index()
train_cards = train_cards.reset_index()
test_cards = test_cards.reset_index()
train_turncounter = train_turncounter.reset_index()
test_turncounter = test_turncounter.reset_index()

(211, 3) (19, 3)
(142, 3) (11, 3)
(186, 3) (20, 3)


## tonks stuff

In [12]:
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.utils.data import Dataset, DataLoader

In [13]:
from tonks import MultiTaskLearner, MultiDatasetLoader


I0112 22:47:48.034691  3368 file_utils.py:35] PyTorch version 1.2.0 available.


In [14]:
import numpy as np
from PIL import Image
import torch
from torch.utils.data import Dataset
from torchvision import transforms

imagenet_rgb_means = [0.485, 0.456, 0.406]
imagenet_rgb_std = [0.229, 0.224, 0.225]
resnet_img_size = 224

full_img_transforms = {
    'train': transforms.Compose([
        transforms.Resize((256, 256)),
        transforms.RandomResizedCrop(resnet_img_size),
        transforms.ToTensor(),
        transforms.Normalize(imagenet_rgb_means, imagenet_rgb_std)
    ]),
    'val': transforms.Compose([
        transforms.Resize((resnet_img_size, resnet_img_size)),
        transforms.ToTensor(),
        transforms.Normalize(imagenet_rgb_means, imagenet_rgb_std)
    ]),
}


class FGOImageDataset(Dataset):
    """
    Load data specifically for use with a image models

    Parameters
    ----------
    x: pandas Series
        file paths to stored images
    y: list
        A list of dummy-encoded categories
        For instance, y might be [0,1,2,0] for a 3 class problem with 4 samples
    transform: str or list of pytorch transforms
        specifies how to preprocess the full image for an attribute image model
        To use built in tonks image transforms use strings `train` or `val`
        To use custom transformations supply a list of pytorch transforms.
    crop_transform: str or list of pytorch transforms
        specifies how to preprocess the center cropped image for an attribute image model
        To use built in tonks image transforms use strings `train` or `val`
        To use custom transformations supply a list of pytorch transforms.
    """
    def __init__(self,
                 x,
                 y,
                 transform='train'):
        self.x = x
        self.y = y

        if transform == 'train' or 'val':
            self.transform = full_img_transforms[transform]

    def __getitem__(self, index):
        """Return tuple of images as pytorch tensors and and tensor of labels"""
        label = self.y[index]
        full_img = Image.open(self.x[index]).convert('RGB')

        full_img = self.transform(full_img)

        label = torch.from_numpy(np.array(label)).float()

        return full_img, label

    def __len__(self):
        return len(self.x)


In [15]:
'''

train_attack,test_attack = train_test_split(attack_dat,.9)
print(train_attack.shape,test_attack.shape)

train_cards,test_cards = train_test_split(cards_dat,.9)
print(train_cards.shape,test_cards.shape)

train_turncounter,test_turncounter = train_test_split(turncounter_dat,.9)
print(train_turncounter.shape,test_turncounter.shape)

'''
attack_train_dataset = FGOImageDataset(
    x=train_attack['file_path'],
    y=train_attack['label_cat'],
    transform='train')
attack_test_dataset = FGOImageDataset(
    x=test_attack['file_path'],
    y=test_attack['label_cat'],
    transform='val')


cards_train_dataset = FGOImageDataset(
    x=train_cards['file_path'],
    y=train_cards['label_cat'],
    transform='train')
cards_test_dataset = FGOImageDataset(
    x=test_cards['file_path'],
    y=test_cards['label_cat'],
    transform='val')


turncounter_train_dataset = FGOImageDataset(
    x=train_turncounter['file_path'],
    y=train_turncounter['label_cat'],
    transform='train')
turncounter_test_dataset = FGOImageDataset(
    x=test_turncounter['file_path'],
    y=test_turncounter['label_cat'],
    transform='val')


In [31]:
batch_size = 16
train_dataloaders_dict = {
    'attack': DataLoader(attack_train_dataset, batch_size=batch_size, shuffle=True, num_workers=0),
    'cards': DataLoader(cards_train_dataset, batch_size=batch_size, shuffle=True, num_workers=0),
    'turncounter': DataLoader(turncounter_train_dataset, batch_size=batch_size, shuffle=True, num_workers=0)
}

valid_dataloaders_dict = {
   'attack': DataLoader(attack_test_dataset, batch_size=batch_size, shuffle=True, num_workers=0),
    'cards': DataLoader(cards_test_dataset, batch_size=batch_size, shuffle=True, num_workers=0),
    'turncounter': DataLoader(turncounter_test_dataset, batch_size=batch_size, shuffle=True, num_workers=0)
}

In [32]:

TrainLoader = MultiDatasetLoader(loader_dict=train_dataloaders_dict)
print(len(TrainLoader))

ValidLoader = MultiDatasetLoader(loader_dict=valid_dataloaders_dict, shuffle=False)
print(len(ValidLoader))

35
5


In [33]:
new_task_dict = {
    'attack': attack_dat['label_cat'].nunique(),
    'cards': cards_dat['label_cat'].nunique(),
    'turncounter': turncounter_dat['label_cat'].nunique(),
}

In [34]:
new_task_dict

{'attack': 2, 'cards': 3, 'turncounter': 3}

In [35]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)


cuda:0


In [57]:
import copy
from pathlib import Path

import torch
import torch.nn as nn
from torchvision import models as torch_models

from tonks.vision.helpers import _dense_block, _Identity


class ResnetForMultiTaskClassification(nn.Module):
    """
    Pytorch image attribute model. This model allows you to load
    in some pretrained tasks in addition to creating new ones.

    Examples
    --------
    To instantiate a completely new instance of ResnetForMultiTaskClassification
    and load the weights into this architecture you can set `pretrained` to True

    ```
        model = ResnetForMultiTaskClassification(
            new_task_dict=new_task_dict,
            load_pretrained_resnet = True
        )

        DO SOME TRAINING

        model.save(SOME_FOLDER, SOME_MODEL_ID)
    ```

    To instantiate an instance of ResnetForMultiTaskClassification that has layers for
    pretrained tasks and new tasks, you would do the following:
    ```
        model = ResnetForMultiTaskClassification(
            pretrained_task_dict=pretrained_task_dict,
            new_task_dict=new_task_dict
        )

        model.load(SOME_FOLDER, SOME_MODEL_DICT)

        DO SOME TRAINING
    ```

    Parameters
    ----------
    pretrained_task_dict: dict
        dictionary mapping each pretrained task to the number of labels it has
    new_task_dict: dict
        dictionary mapping each new task to the number of labels it has
    load_pretrained_resnet: boolean
        flag for whether or not to load in pretrained weights for ResNet50.
        useful for the first round of training before there are fine tuned weights
    """

    def __init__(self, pretrained_task_dict=None, new_task_dict=None, load_pretrained_resnet=False):
        super(ResnetForMultiTaskClassification, self).__init__()

        self.resnet = torch_models.resnet50(pretrained=load_pretrained_resnet)
        self.resnet.fc = _Identity()

        if pretrained_task_dict is not None:
            pretrained_layers = {}
            for key, task_size in pretrained_task_dict.items():
                pretrained_layers[key] = nn.Linear(2048, task_size)
            self.pretrained_classifiers = nn.ModuleDict(pretrained_layers)
        if new_task_dict is not None:
            new_layers = {}
            for key, task_size in new_task_dict.items():
                new_layers[key] = nn.Linear(2048, task_size)
            self.new_classifiers = nn.ModuleDict(new_layers)

    def forward(self, x):
        """
        Defines forward pass for image model

        Parameters
        ----------
        x: dict of image tensors containing tensors for
        full and cropped images. the full image tensor
        has the key 'full_img' and the cropped tensor has
        the key 'crop_img'

        Returns
        ----------
        A dictionary mapping each task to its logits
        """
        full_img = self.resnet(x)

        #full_crop_combined = torch.cat((full_img, crop_img), 1)

        #dense_layer_output = self.dense_layers(full_crop_combined)

        logit_dict = {}
        if hasattr(self, 'pretrained_classifiers'):
            for key, classifier in self.pretrained_classifiers.items():
                logit_dict[key] = classifier(full_img)
        if hasattr(self, 'new_classifiers'):
            for key, classifier in self.new_classifiers.items():
                logit_dict[key] = classifier(full_img)

        return logit_dict

    def freeze_core(self):
        """Freeze all core model layers"""
        for param in self.resnet.parameters():
            param.requires_grad = Fals

    def freeze_all_pretrained(self):
        """Freeze pretrained classifier layers and core model layers"""
        self.freeze_core()
        if hasattr(self, 'pretrained_classifiers'):
            for param in self.pretrained_classifiers.parameters():
                param.requires_grad = False
        else:
            print('There are no pretrained_classifier layers to be frozen.')

    def unfreeze_pretrained_classifiers(self):
        """Unfreeze pretrained classifier layers"""
        if hasattr(self, 'pretrained_classifiers'):
            for param in self.pretrained_classifiers.parameters():
                param.requires_grad = True
        else:
            print('There are no pretrained_classifier layers to be unfrozen.')

    def unfreeze_pretrained_classifiers_and_core(self):
        """Unfreeze pretrained classifiers and core model layers"""
        for param in self.resnet.parameters():
            param.requires_grad = True
        self.unfreeze_pretrained_classifiers()

    def save(self, folder, model_id):
        """
        Saves the model state dicts to a specific folder.
        Each part of the model is saved separately to allow for
        new classifiers to be added later.

        Note: if the model has `pretrained_classifiers` and `new_classifers`,
        they will be combined into the `pretrained_classifiers_dict`.

        Parameters
        ----------
        folder: str or Path
            place to store state dictionaries
        model_id: int
            unique id for this model

        Side Effects
        ------------
        saves two files:
            - folder / f'resnet_dict_{model_id}.pth'
            - folder / f'pretrained_classifiers_dict_{model_id}.pth'
        """
        if not hasattr(self, 'pretrained_classifiers'):
            classifiers_to_save = copy.deepcopy(self.new_classifiers)
        else:
            # Pytorch's update method isn't working because it doesn't think ModuleDict is a Mapping
            classifiers_to_save = copy.deepcopy(self.pretrained_classifiers)
            if hasattr(self, 'new_classifiers'):
                for key, module in self.new_classifiers.items():
                    classifiers_to_save[key] = module

        folder = Path(folder)
        folder.mkdir(parents=True, exist_ok=True)

        torch.save(
            self.resnet.state_dict(),
            folder / f'resnet_dict_{model_id}.pth'
        )
        torch.save(
            classifiers_to_save.state_dict(),
            folder / f'pretrained_classifiers_dict_{model_id}.pth'
        )

    def load(self, folder, model_id):
        """
        Loads the model state dicts from a specific folder.

        Parameters
        ----------
        folder: str or Path
            place where state dictionaries are stored
        model_id: int
            unique id for this model

        Side Effects
        ------------
        loads from two files:
            - folder / f'resnet_dict_{model_id}.pth'
            - folder / f'pretrained_classifiers_dict_{model_id}.pth'
        """
        #folder = Path(folder)

        if torch.cuda.is_available():
            self.resnet.load_state_dict(torch.load(folder +'/'+ f'resnet_dict_{model_id}.pth'))
            
            self.pretrained_classifiers.load_state_dict(
                torch.load(folder / f'pretrained_classifiers_dict_{model_id}.pth')
            )
        else:
            self.resnet.load_state_dict(
                torch.load(
                    folder / f'resnet_dict_{model_id}.pth',
                    map_location=lambda storage,
                    loc: storage
                )
            )
        
            self.pretrained_classifiers.load_state_dict(
                torch.load(
                    folder / f'pretrained_classifiers_dict_{model_id}.pth',
                    map_location=lambda storage,
                    loc: storage
                )
            )

    def export(self, folder, model_id, model_name=None):
        """
        Exports the entire model state dict to a specific folder.
        Note: if the model has `pretrained_classifiers` and `new_classifiers`,
        they will be combined into the `pretrained_classifiers` attribute before being saved.

        Parameters
        ----------
        folder: str or Path
            place to store state dictionaries
        model_id: int
            unique id for this model
        model_name: str (defaults to None)
            Name to store model under, if None, will default to `multi_task_bert_{model_id}.pth`

        Side Effects
        ------------
        saves one file:
            - folder / model_name
        """
        if hasattr(self, 'new_classifiers'):
            hold_new_classifiers = copy.deepcopy(self.new_classifiers)
        else:
            hold_new_classifiers = None

        hold_pretrained_classifiers = None
        if not hasattr(self, 'pretrained_classifiers'):
            self.pretrained_classifiers = copy.deepcopy(self.new_classifiers)
        else:
            hold_pretrained_classifiers = copy.deepcopy(self.pretrained_classifiers)
            # Pytorch's update method isn't working because it doesn't think ModuleDict is a Mapping
            if hasattr(self, 'new_classifiers'):
                for key, module in self.new_classifiers.items():
                    self.pretrained_classifiers[key] = module
        if hasattr(self, 'new_classifiers'):
            del self.new_classifiers

        if model_name is None:
            model_name = f'multi_task_resnet_{model_id}.pth'

        folder = Path(folder)
        folder.mkdir(parents=True, exist_ok=True)

        torch.save(
            self.state_dict(),
            folder / model_name
        )
        if hold_pretrained_classifiers is not None:
            self.pretrained_classifiers = hold_pretrained_classifiers
        else:
            del self.pretrained_classifiers

        if hold_new_classifiers is not None:
            self.new_classifiers = hold_new_classifiers


In [37]:
model = ResnetForMultiTaskClassification(
    new_task_dict=new_task_dict,
    load_pretrained_resnet=True
)

In [38]:

loss_function = nn.CrossEntropyLoss()

lr_last = 1e-2
lr_main = 1e-4

optimizer = optim.Adam([
    {'params': model.resnet.parameters(), 'lr': lr_main},
    {'params': model.new_classifiers.parameters(), 'lr': lr_last},
    
])

exp_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size= 4, gamma= 0.1)

In [39]:
learn = MultiTaskLearner(model, TrainLoader, ValidLoader, new_task_dict)


In [40]:
learn.fit(
    num_epochs=10,
    loss_function=loss_function,
    scheduler=exp_lr_scheduler,
    step_scheduler_on_batch=False,
    optimizer=optimizer,
    device=device,
    best_model=True
)

train_loss,val_loss,attack_train_loss,attack_val_loss,attack_acc,cards_train_loss,cards_val_loss,cards_acc,turncounter_train_loss,turncounter_val_loss,turncounter_acc,time
5.813186,20.480856,3.362564,6.050823,0.947368,9.469371,0.002709,1.000000,5.801910,45.452368,0.400000,00:05
0.897026,2.321326,0.795331,5.901299,0.947368,1.076745,0.207065,0.909091,0.875185,0.083194,0.950000,00:04
0.384623,0.872330,0.380775,2.295447,0.947368,0.417688,0.000005,1.000000,0.363745,0.000149,1.000000,00:04
0.269163,0.718666,0.317839,1.887572,0.947368,0.231201,0.000438,1.000000,0.242925,0.003230,1.000000,00:04
0.152203,0.717088,0.124498,1.885724,0.947368,0.133552,0.000555,1.000000,0.197869,0.000977,1.000000,00:04
0.101304,0.735356,0.081647,1.934233,0.947368,0.092086,0.000630,1.000000,0.130639,0.000523,1.000000,00:04
0.100545,0.654007,0.133052,1.718293,0.947368,0.046010,0.000208,1.000000,0.105303,0.002526,1.000000,00:04
0.196440,0.450437,0.185399,1.178312,0.947368,0.254179,0.000312,1.000000,0.164883,0.006525,1.000000,00:04
0.091072,0.368469,0.088733,0.968099,0.947368,0.016666,0.000109,1.000000,0.150530,0.001418,1.000000,00:04
0.148534,0.358295,0.145955,0.939197,0.947368,0.071307,0.000192,1.000000,0.210419,0.003395,1.000000,00:04


Epoch 9 best model saved with loss of 0.3582952976046363


In [41]:
model.save(folder='models/', model_id='fgo_model1')

model.export(folder='models/', model_id='fgo_model1')


In [74]:
import copy
from pathlib import Path

import torch
import torch.nn as nn
from torchvision import models as torch_models

from tonks.vision.helpers import _dense_block, _Identity


class ResnetForMultiTaskClassification(nn.Module):
    """
    Pytorch image attribute model. This model allows you to load
    in some pretrained tasks in addition to creating new ones.

    Examples
    --------
    To instantiate a completely new instance of ResnetForMultiTaskClassification
    and load the weights into this architecture you can set `pretrained` to True

    ```
        model = ResnetForMultiTaskClassification(
            new_task_dict=new_task_dict,
            load_pretrained_resnet = True
        )

        DO SOME TRAINING

        model.save(SOME_FOLDER, SOME_MODEL_ID)
    ```

    To instantiate an instance of ResnetForMultiTaskClassification that has layers for
    pretrained tasks and new tasks, you would do the following:
    ```
        model = ResnetForMultiTaskClassification(
            pretrained_task_dict=pretrained_task_dict,
            new_task_dict=new_task_dict
        )

        model.load(SOME_FOLDER, SOME_MODEL_DICT)

        DO SOME TRAINING
    ```

    Parameters
    ----------
    pretrained_task_dict: dict
        dictionary mapping each pretrained task to the number of labels it has
    new_task_dict: dict
        dictionary mapping each new task to the number of labels it has
    load_pretrained_resnet: boolean
        flag for whether or not to load in pretrained weights for ResNet50.
        useful for the first round of training before there are fine tuned weights
    """

    def __init__(self, pretrained_task_dict=None, new_task_dict=None, load_pretrained_resnet=False):
        super(ResnetForMultiTaskClassification, self).__init__()

        self.resnet = torch_models.resnet50(pretrained=load_pretrained_resnet)
        self.resnet.fc = _Identity()

        if pretrained_task_dict is not None:
            pretrained_layers = {}
            for key, task_size in pretrained_task_dict.items():
                pretrained_layers[key] = nn.Linear(2048, task_size)
            self.pretrained_classifiers = nn.ModuleDict(pretrained_layers)
        if new_task_dict is not None:
            new_layers = {}
            for key, task_size in new_task_dict.items():
                new_layers[key] = nn.Linear(2048, task_size)
            self.new_classifiers = nn.ModuleDict(new_layers)

    def forward(self, x):
        """
        Defines forward pass for image model

        Parameters
        ----------
        x: dict of image tensors containing tensors for
        full and cropped images. the full image tensor
        has the key 'full_img' and the cropped tensor has
        the key 'crop_img'

        Returns
        ----------
        A dictionary mapping each task to its logits
        """
        full_img = self.resnet(x)

        #full_crop_combined = torch.cat((full_img, crop_img), 1)

        #dense_layer_output = self.dense_layers(full_crop_combined)

        logit_dict = {}
        if hasattr(self, 'pretrained_classifiers'):
            for key, classifier in self.pretrained_classifiers.items():
                logit_dict[key] = classifier(full_img)
        if hasattr(self, 'new_classifiers'):
            for key, classifier in self.new_classifiers.items():
                logit_dict[key] = classifier(full_img)

        return logit_dict

    

imsize = 224

loader = transforms.Compose([transforms.Resize((imsize,imsize)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])

scale=transforms.Compose([transforms.Resize((224,224)),transforms.ToTensor()])
from torch.autograd import Variable

def image_loader(image_name):
    """load image, returns cuda tensor"""
    image = loader(image_name).float()
    image = Variable(image, requires_grad=True)
    image = image.unsqueeze(0)  #this is for VGG, may not be needed for ResNet
    return image.cuda()  #assumes that you're using GPU


from PIL import Image
img = Image.open('fgo_image_datasets/attack/attack/a1.JPG')
img = image_loader(img)


tensor([[[[-0.7822, -0.0801,  1.0502,  ..., -1.0219, -1.0904, -1.1247],
          [-0.3883,  0.1939,  1.1187,  ..., -1.1247, -1.1932, -1.2274],
          [ 0.4508,  0.7591,  1.2557,  ..., -1.3473, -1.3987, -1.4158],
          ...,
          [-1.1760, -1.2103, -1.2617,  ..., -1.7069, -1.5699, -1.4843],
          [-1.3130, -1.3473, -1.4158,  ..., -1.5357, -1.3644, -1.2788],
          [-1.3815, -1.4158, -1.4843,  ..., -1.4500, -1.2788, -1.1760]],

         [[-0.4776,  0.2752,  1.4307,  ..., -1.1429, -1.2129, -1.2479],
          [-0.0749,  0.5553,  1.5007,  ..., -1.2479, -1.3179, -1.3529],
          [ 0.7829,  1.1331,  1.6758,  ..., -1.4755, -1.5280, -1.5455],
          ...,
          [-1.6856, -1.7031, -1.7381,  ..., -1.8081, -1.6681, -1.5805],
          [-1.7731, -1.7906, -1.8431,  ..., -1.6331, -1.4580, -1.3704],
          [-1.8081, -1.8431, -1.8957,  ..., -1.5455, -1.3704, -1.2654]],

         [[-0.8110, -0.0790,  1.0714,  ..., -1.0201, -1.0898, -1.1247],
          [-0.4101,  0.1825,  

In [58]:
new_task_dict
model2 = ResnetForMultiTaskClassification(pretrained_task_dict=new_task_dict,
    load_pretrained_resnet=False)

In [77]:
from pathlib import Path
model2.load_state_dict(torch.load("models/multi_task_resnet_fgo_model1.pth"))
model2 = model2.to(device)
model2.eval()

ResnetForMultiTaskClassification(
  (resnet): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample)

In [114]:
softmax = nn.Softmax(1)


def get_preds(image,category):
    if category == 'attack':
        labels = { 0:'attack', 1:'not_attack'}
    if category == 'cards':
        labels = { 0:'arts', 1:'buster',2:'quick'}
    if category == 'turncounter':
        labels = {0:'one', 1:'three',2: 'two'}
    full_preds = model2(image)
    preds_out = softmax(full_preds[category])
    label_out = labels[preds_out.cpu().data.numpy().argmax()]
    
    return label_out


get_preds(img,'attack')

'not_attack'

In [115]:
img = Image.open('fgo_image_datasets/attack/not_attack/a1.JPG')
img = image_loader(img)
get_preds(img,'attack')

'not_attack'

In [116]:
img = Image.open('fgo_image_datasets/attack/attack/a1.JPG')
img = image_loader(img)
get_preds(img,'attack')

'attack'

In [117]:
#buster
img = Image.open('fgo_image_datasets/cards/buster/b1.JPG')
img = image_loader(img)
get_preds(img,'cards')

'buster'

In [118]:
img = Image.open('fgo_image_datasets/turncounter/three/1.JPG')
img = image_loader(img)
get_preds(img,'turncounter')

'three'

In [119]:
img = Image.open('fgo_image_datasets/turncounter/one/1.JPG')
img = image_loader(img)
get_preds(img,'turncounter')

'one'